In [1]:
import os
import nltk

In [ ]:
import re
!pip install pymorphy2

In [15]:
import pandas as pd

In [ ]:
os.listdir("corpus")

In [ ]:
!wget https://raw.githubusercontent.com/dhhse/dh2020/master/data/stop_ru.txt

In [7]:
with open('stop_ru.txt', 'r') as stop_list_file:
    stop_list_as_list = stop_list_file.readlines()
    stop_words = [line.strip() for line in stop_list_as_list]

In [9]:
def read_book(path_to_file: str):
  path_to_file = 'corpus/' + path_to_file
  with open(path_to_file, 'r') as book: 
    book = book.read()
  return book

In [ ]:
from nltk.tokenize import word_tokenize #токенизация
nltk.download('punkt')

In [10]:
def tokenize (book: str, stop_words: list):
  book_tokenized = word_tokenize(book.lower())
  book_no_punct = [word for word in book_tokenized if word[0].isalpha()]
  tokens = [word for word in book_no_punct  if word not in stop_words]
  return tokens

In [18]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [22]:
def lemmatize (tokens):
  lemmas = [morph.parse(word)[0].normal_form for word in tokens]
  return lemmas

In [ ]:
!wget https://raw.githubusercontent.com/Dekromatia/Graph_animals/main/animal_list.txt

In [24]:
with open('animal_list.txt', 'r') as animal_file:
    animal_as_list = animal_file.readlines()
    animal_words = [line.strip() for line in animal_as_list]
    animals = [word.lower() for word in animal_words]

In [33]:
def find_animals (lemmas, animals):
  animals_book = [word for word in lemmas if word in animals]
  book_animals = list(set(animals_book))
  #with open ('Akimushkit_Chem_krolik_na_zayca_ne_pohozh_animals.txt', 'w') as f:
  #for x in unic_animals_book:
  # f.write(x +"\n")
  return book_animals

In [35]:
paths = os.listdir('corpus')
book2animals = {} # СЛОВАРЬ №1 ключ - название книги, значение - список животных
for path in paths:
  book = read_book(path)
  tokens = tokenize(book, stop_words)
  lemmas = lemmatize(tokens) # лемматизируем
  book_animals = find_animals(lemmas, animals) # animals берем из файла в виде списка слов

  path = path.strip('.txt')
  book2animals[path] = book_animals #создаем словарь книга-ключ; список животных - значение

In [37]:
#меняем местами ключ и значение, тепрь животное - ключ а список книг - значение
def aggregate_books(book2animals):
  animal2books = {}
  for book, animals in book2animals.items():
    for animal in animals:
      if animal in animal2books:
        animal2books[animal].append(book)
      if animal not in animal2books:
        animal2books[animal]=[book]       
  return animal2books 

In [38]:
#СЛОВАРЬ №2
animal2books = aggregate_books(book2animals)

In [ ]:
animal2books

In [84]:
with open ('for_easy_linavis.txt', 'w') as f:
  for key, value  in animal2books.items():
    if len(value)>1:
      print(len(value))
      f.write("#" + key +'\n')
      for name in value:
        f.write(name + '\n')